In [ ]:
import os
from avapi.carla import CarlaScenesManager, get_splits_scenes


cpath = os.path.join("/data/shared/CARLA/multi-agent-v1/")
# cpath = "../examples/sim_results"
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes["val"][0])
print(f"{len(CDM)} frames")

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import MMDetObjectDetector3D


# init detectors
detector_2d =  MMDetObjectDetector2D(model="fasterrcnn", dataset="carla-joint", threshold=0.5)
detector_3d = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle", threshold=0.5, epoch=1)
# detector_3d_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure", threshold=0.2, epoch=3)

In [ ]:
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_lidar_on_image, show_image_with_boxes

# load data
agent = 1
frame_idx = 100
camera_sensor = "camera-0"
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)

# 2d detector
objs_2d = detector_2d(img)
if agent < 5:
    objs_3d = detector_3d(pc)
else:
    objs_3d = detector_3d_inf(pc)

# show_lidar_bev_with_boxes(pc=pc, boxes=objs)
# show_lidar_on_image(img=img, pc=pc, boxes=objs)
# show_image_with_boxes(img=img, boxes=objs, inline=True, return_image=False)

print([obj.box.position.x[2] for obj in objs_3d])

show_lidar_bev_with_boxes(pc=pc, boxes=objs)
show_lidar_bev_with_boxes(pc=pc, boxes=objs_3d)
show_image_with_boxes(img=img, boxes=objs_3d, inline=True, return_image=False)


## Test projection operation

In [ ]:
# load data
agent = 6
frame_idx = 5
camera_sensor = "camera-0"
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)


In [ ]:
import numpy as np
from avstack.calibration import LidarCalibration
from avstack.geometry import transform_orientation

calib_new = LidarCalibration(reference=pc.reference.get_ground_projected_reference())
pc_new = pc.project(calib_new)
boxes_new = [obj.box.change_reference(pc_new.reference, inplace=False) for obj in objs]

print(transform_orientation(pc.reference.reference.q, 'quat', 'euler') * 180/np.pi)
print(transform_orientation(pc_new.reference.q, 'quat', 'euler') * 180/np.pi)

In [ ]:
boxes_new

In [ ]:
show_lidar_bev_with_boxes(pc=pc, boxes=objs)
show_lidar_bev_with_boxes(pc=pc_new, boxes=boxes_new)

In [ ]:
np.min(pc.data.x, axis=0)
np.max(pc.data.x, axis=0)